In this notebook, we show examples for using the Structure Learning Algorithms in pgmpy. Currently, pgmpy has implementation of 3 main algorithms:
1. PC with stable and parallel variants.
2. Hill-Climb Search
3. Exhaustive Search

For PC the following conditional independence test can be used:
1. Chi-Square test (https://en.wikipedia.org/wiki/Chi-squared_test)
2. Pearsonr (https://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression)
3. G-squared (https://en.wikipedia.org/wiki/G-test)
4. Log-likelihood (https://en.wikipedia.org/wiki/G-test)
5. Freeman-Tuckey (Read, Campbell B. "Freeman—Tukey chi-squared goodness-of-fit statistics." Statistics & probability letters 18.4 (1993): 271-278.)
6. Modified Log-likelihood
7. Neymann (https://en.wikipedia.org/wiki/Neyman%E2%80%93Pearson_lemma)
8. Cressie Read (Cressie, Noel, and Timothy RC Read. "Multinomial goodness‐of‐fit tests." Journal of the Royal Statistical Society: Series B (Methodological) 46.3 (1984): 440-464)
9. Power Divergence (Cressie, Noel, and Timothy RC Read. "Multinomial goodness‐of‐fit tests." Journal of the Royal Statistical Society: Series B (Methodological) 46.3 (1984): 440-464.)

For Hill-Climb and Exhausitive Search the following scoring methods can be used:
1. K2 Score
2. BDeu Score
3. Bic Score

## Generate some data

In [1]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score

from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

In [2]:
model = get_example_model('alarm')
samples = BayesianModelSampling(model).forward_sample(size=int(1e3))
samples.head()

Generating for node: VENTMACH:   0%|          | 0/37 [00:00<?, ?it/s] /home/ankur/pgmpy/examples/pgmpy/factors/discrete/DiscreteFactor.py:517: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warn(
Generating for node: CVP: 100%|██████████| 37/37 [00:00<00:00, 150.14it/s]         


,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,INTUBATION,PULMEMBOLUS,SHUNT,PAP,FIO2,KINKEDTUBE,...,HRBP,LVFAILURE,HISTORY,HYPOVOLEMIA,STROKEVOLUME,CO,BP,LVEDVOLUME,PCWP,CVP
0,NORMAL,NORMAL,TRUE,ZERO,NORMAL,FALSE,NORMAL,NORMAL,NORMAL,FALSE,...,HIGH,FALSE,FALSE,FALSE,LOW,NORMAL,LOW,NORMAL,NORMAL,NORMAL
1,NORMAL,NORMAL,FALSE,LOW,NORMAL,FALSE,NORMAL,NORMAL,LOW,FALSE,...,HIGH,TRUE,TRUE,FALSE,LOW,LOW,LOW,LOW,LOW,LOW
2,HIGH,HIGH,FALSE,HIGH,NORMAL,FALSE,NORMAL,NORMAL,NORMAL,FALSE,...,HIGH,FALSE,FALSE,FALSE,NORMAL,HIGH,LOW,NORMAL,NORMAL,NORMAL
3,NORMAL,LOW,FALSE,ZERO,ONESIDED,FALSE,HIGH,NORMAL,NORMAL,FALSE,...,HIGH,FALSE,FALSE,FALSE,NORMAL,HIGH,HIGH,NORMAL,NORMAL,NORMAL
4,NORMAL,NORMAL,FALSE,LOW,ONESIDED,FALSE,HIGH,NORMAL,NORMAL,FALSE,...,HIGH,FALSE,FALSE,FALSE,NORMAL,HIGH,HIGH,NORMAL,LOW,NORMAL


In [3]:
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_matrix(estimated_model.to_undirected(), nodelist=nodes, weight=None)
    true_adj = nx.to_numpy_matrix(true_model.to_undirected(), nodelist=nodes, weight=None)
    
    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

## Learn the model structure using Hill-Climb Search

In [6]:
"""
*理论知识：
score评分评的是一个模型和所给数据集的贴合程度：

一个结构的贝叶斯评分：后验概率 log p(结构|数据)
log p(结构|数据)= log p(数据|结构) + log(结构)

p(结构)是结构先验分布，一般假设为均匀分布

p(数据|结构)是边缘似然函数，展开=对该结构的参数进行积分(p(数据|结构，参数)*p(参数|结构))

进行一步一步推理后得到：
CH评分：
l(结构|数据)= 所有节点i的和（所有父亲j的和（   loggamma(aij*)  
                                         -loggamma(aij*+mij*)
                                         +所有节点可能取值k的和（loggamma(aijk)
                                                          -loggamma(aijk+mijk)                                             
                                                        ）
                                         ））
                            
注意：
1、由于CH评分是可以分解的，所以可以先对每个节点求分，最后相加。                                       
2、在k2算法中，为简化模型计算，
   假设所有参数先验分布都是均匀分布：即aijk和aij*都是0

"""


"""
*实践：
用了k2score但其实不是k2算法，只是k2算法中的评分那部分
k2score中local_score函数：
计算每一个节点受其可能父节点影响的分值：
 （1）数据准备
        var_states该节点可能的取值
                        = self.state_names[variable]
        var_cardinality该节点可能取值的个数
                        = len(var_states)
        state_counts该节点可能取值与父亲们的可能取值组合构成的表
                        = self.state_counts(variable, parents)
        num_parents_states父亲们可能的组合个数
                        = float(state_counts.shape[1])
        counts该节点可能取值与父亲们的可能取值组合构成的表转为矩阵
                        = np.asarray(state_counts)
        log_gamma_counts将counts初始化为0
                        = np.zeros_like(counts, dtype=np.float_)

   （2）计算 log(gamma(counts + 1))即公式中的log(gamma(aij*+mijk*))
        gammaln(counts + 1, out=log_gamma_counts)

    (3)计算公式中的  对于该节点所有可能取值k的和（ log(gamma(mijk+aijk)) ）
        log_gamma_conds首先计算mijk对于k求和也就是counts的每一列求和
                         = np.sum(counts, axis=0, dtype=np.float_)
        aijk对于k求和，也就是counts表的每一列的1相加，
                       结果都是该节点可能取值个数var_cardinality
        gammaln(log_gamma_conds + var_cardinality, out=log_gamma_conds)
        
   （4）最后合起来计算，对所有父亲求和，也就是np.sum()
       首先，注意符号，与公式中相反
       其次，省略log(gamma(aij*))的计算，因为所有结构计算出来都相同，因为aij*都为1
       最后，对公式中  所有节点可能取值k的和（loggamma(aijk)) 的计算
            就是logamma(var_cardinality)，
            然后要对所有父亲求和，因此乘以父亲组合的个数num_parantes-states
                                                 
        score = (
            np.sum(log_gamma_counts)
            - np.sum(log_gamma_conds)
            + num_parents_states * lgamma(var_cardinality)
        )
        
对每个节点的分值计算完毕，由StructureScore中的score函数调用local_score计算总的：
        for node in model.nodes():
            score += self.local_score(node, model.predecessors(node))

"""  


scoring_method = K2Score(data=samples)



"""
爬山法:
从 初始的 无边 模型 出发开始搜索：
每一步使用搜索算子(search operator)对当前模型(current model)进行局部修改，
得到一系列候选模型(candidate model)，计算每个候选模型的评分，并将最优模型与当前模型比较，
若是最优候选模型的评分大，则以它为下一个当前模型，继续搜索
否则就停止搜索，并返回当前模型。

搜索算子有3个：加边、减边、转边
注意：加边和转变不能生成有向圈

爬山法可以使用任何评分函数，这里使用k2中的评分方法。

"""

est = HillClimbSearch(data=samples)

"""
实践：
HillClimbSearch类继承StructureEstimator。
其中estimate函数：
参数： 
scoring_method：str3选1： "k2score"，"bdeuscore"，"bicscore"
                自定义StructureScore
start_dag:初始的结构
fixed_edges：算法中一直存在且不变的边
tabu_length：最后tabu_length个图禁止转边
max_indegree：最大父亲节点个数
black_list：禁止加入的边
white_list：允许加入的边
epsilon：评分增加幅度小于epsilon，可以返回了
max_iter：最大允许迭代次数，超过返回

1、做好准备工作：
  （1）检查评分方法3选1或自定义的
      
  （2）检车初始结构start_dag，只加入点
          if start_dag is None:
            start_dag = DAG()
            start_dag.add_nodes_from(self.variables)
            
   (3)检查fixed_edges
      加入fixed_edges到start_dag中
      
  （4）准备好white_list和black_list
  
  （5）初始化max_indegree为inf、current_model为start_dag，tabu_list
  
2、开始迭代，每轮迭代找到最高分的作为下一个current_model
每一轮迭代中，调用_legal_operation函数：
  （1）获得所有加边的合法操作
      potential_new_edges = (
            set(permutations(self.variables, 2))#permutations获得所有排列
            - set(model.edges())#删掉已存在的边
            - set([(Y, X) for (X, Y) in model.edges()])#删掉已存在的边的转向边
        )
      如果加入potential_new_edges中的一条边（x,y）不会导致有向圈
         令operation=("+"(x,y))
         如果operation不在tabu_list
         并且不在black_list
         并且在white_list
            old_parents设为Y原来父亲
            new_parents设为y原来父亲+x
            如果new_parents个数<max_indegree
              score_delta = score(Y, new_parents) - score(Y, old_parents)
              yield (operation, score_delta)
      
  
  （2）获得所有合法删边操作
          for (X, Y) in model.edges():
            operation = ("-", (X, Y))
            if (operation not in tabu_list) and ((X, Y) not in fixed_edges):
                old_parents = model.get_parents(Y)
                new_parents = old_parents[:]
                new_parents.remove(X)
                score_delta = score(Y, new_parents) - score(Y, old_parents)
                yield (operation, score_delta)
  
  （3）获得所有合法转边操作(flip edges)
      如果不形成环：（xy之间简单路长度大于2则有环）：
      如果。。。。。。
       score_delta = (
                            score(X, new_X_parents)
                            + score(Y, new_Y_parents)
                            - score(X, old_X_parents)
                            - score(Y, old_Y_parents)
                        )  
  
每一迭代中，
   获取best_operation, best_score_delta 
     =max(elf._legal_operations(
                    current_model,
                    score_fn,
                    tabu_list,
                    max_indegree,
                    black_list,
                    white_list,
                    fixed_edges,
                ),
                key=lambda t: t[1],
                default=(None, None),
        )
    若best_operation若空或best_score_delta<epsilon，返回
    分情况讨论+/-/flip，分别对current_model进行相应操作
    
退出迭代后，返回current_model
         
"""
estimated_model = est.estimate(scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4))


"""
f1_score为分类问题的一个衡量指标。
一些多分类问题的机器学习竞赛，常常将F1-score作为最终测评的方法。
它是精确率和召回率的调和平均数，最大为1，最小为0。
这里model作为已知的最佳模型，estimated_model为预测的最佳模型
F1 = 2 * (precision * recall) / (precision + recall)
具体见
https://blog.csdn.net/qq_14997473/article/details/82684300
"""

get_f1_score(estimated_model, model)

  1%|          | 61/10000 [00:36<1:39:54,  1.66it/s]

F1-score for the model skeleton:  0.8076923076923076
